In [ ]:
import oracledb, os, json
from google.cloud import secretmanager
import pandas as pd
import numpy as np
import paramiko
from io import StringIO
from datetime import date
from datetime import timedelta
import datetime

In [ ]:
def get_periode():
    """
    henter periode for the tidligere måneden eksample--> i dag er 19.04.2022, metoden vil kalkulerer periode aarMaaned eks) '202203'
    :param periode:
    :return: periode
    """
    today = datetime.date.today() # dato for idag 2022-04-19
    first = today.replace(day=1) # dato for første dag i måneden 2022-04-01
    lastMonth = first - datetime.timedelta(days=1) # dato for siste dag i tidligere måneden

    return lastMonth.strftime("%Y%m") # henter bare aar og maaned

periode = get_periode()

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    sftpkey = os.getenv('SFTPKEY'),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
keyfile = StringIO(oracle_secrets['sftpkey'])
mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

# Open a transport
host,port = "a01drvl099.adeo.no",22
transport = paramiko.Transport((host,port))

# Auth    
username= "srv-dv-familie-airflow-sas"
transport.connect(username=username,pkey=mykey)

with paramiko.SFTPClient.from_transport(transport) as sftp:
    sftp.get(f'./inbound/kildefiler/bidrag/BIDRAG_BERM_M{periode}', f'../data/BIDRAG_BERM_M{periode}')
    #print(sftp.listdir(path='.'))

# Close
if sftp: sftp.close()
if transport: transport.close()

In [ ]:
df = pd.read_csv(f"../data/BIDRAG_BERM_M2024{periode}", skip_blank_lines=True, sep='\0', header=None,engine='python')
df['REC_TYPE'] = df[0].apply(lambda x: x[13:15])

filt = (df['REC_TYPE'] == '02')
df_rec_type_2 = df.loc[filt].copy()

kolonne_navn = ["@1   REGDATO         6.",   "@7   SAKSNR   $7.",         "@14  RECTY     2.",     
"@16  bmfnr       $11.",   "@7   NR          $2.",      "@27  VEDTDATO $8.",   
"@35  SKATTFOR      7.",   "@18  fmnd         $2.",      "@20  faar        $2.",
"@42  INNTYP1      $4.",   "@46  INNTYP2      $4.",     "@50  INNTYP3      $4.",
"@54  INNTYP4      $4.",   "@58  INNTYP5      $4.",     "@62  KONTST       $4.",
"@66  INNTBEL1      8.",   "@74  INNTBEL2      8.",     "@82  INNTBEL3      8.",
"@90  INNTBEL4      8.",   "@98  INNTBEL5      8.",     "@106 KONTSBEL      8.",
"@114 BTILRED      $1.",   "@115 BTILBEL       7.",     "@122 BTILFORS     $1.",
"@123 INNTYP1_FO   $4.",   "@127  INNTYP2_FO  $4.",     "@131  INNTYP3_FO  $4.",
"@135  INNTYP4_FO  $4.",   "@139  INNTYP5_FO  $4.",     "@143  KONTST_FO   $4.",
"@147  INNTBEL1_FO  8.",   "@155  INNTBEL2_FO  8.",     "@163  INNTBEL3_FO  8.",
"@171  INNTBEL4_FO  8.",   "@179  INNTBEL5_FO  8.",     "@187 KONTSBEL_FO   8."]

for j in kolonne_navn:
    m = j.split()
    kolonnne = m[1]
    start_pos = int("".join([str(s) for s in m[0] if s.isdigit()])) - 1
    end_pos = int("".join([str(s) for s in m[2] if s.isdigit()])) + start_pos
    #print(f"kolonne er {kolonnne}, starts at pos {start_pos}, ends at {end_pos}")
    df_rec_type_2[kolonnne] = df_rec_type_2[0].apply(lambda x: x[start_pos:end_pos])

df_mottaker_pos = df_rec_type_2[['REGDATO','SAKSNR','RECTY','bmfnr','NR','VEDTDATO','SKATTFOR','fmnd','faar','INNTYP1',
    'INNTYP2','INNTYP3','INNTYP4','INNTYP5','KONTST','INNTBEL1','INNTBEL2','INNTBEL3','INNTBEL4','INNTBEL5','KONTSBEL','BTILRED',
    'BTILBEL','BTILFORS','INNTYP1_FO','INNTYP2_FO','INNTYP3_FO','INNTYP4_FO','INNTYP5_FO','KONTST_FO',
    'INNTBEL1_FO','INNTBEL2_FO', 'INNTBEL3_FO', 'INNTBEL4_FO','INNTBEL5_FO', 'KONTSBEL_FO']]   

df_mottaker_pos = df_mottaker_pos.replace(r'^\s*$', None, regex=True)

user = oracle_secrets['user'] + '[DVH_FAM_BB]'
dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
    with conn.cursor() as cursor:
        rows = [tuple(x) for x in df_mottaker_pos.values]
        cursor.executemany('''INSERT INTO FAM_BB_mottaker_BERM (REGDATO,SAKSNR,RECTY,bmfnr,NR,VEDTDATO,SKATTFOR,fmnd,faar,INNTYP1,
                                INNTYP2,INNTYP3,INNTYP4,INNTYP5,KONTST,INNTBEL1,INNTBEL2,INNTBEL3,INNTBEL4,INNTBEL5,KONTSBEL,BTILRED,
                                BTILBEL,BTILFORS,INNTYP1_FO,INNTYP2_FO,INNTYP3_FO,INNTYP4_FO,INNTYP5_FO,KONTST_FO,
                                INNTBEL1_FO,INNTBEL2_FO, INNTBEL3_FO, INNTBEL4_FO,INNTBEL5_FO, KONTSBEL_FO) 
                                VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36)''',rows)
        conn.commit()